In [11]:
from datasets import load_dataset

ds = load_dataset('xtreme', 'PAN-X.eu')

Using the latest cached version of the dataset since xtreme couldn't be found on the Hugging Face Hub (offline mode is enabled).
Found the latest cached dataset configuration 'PAN-X.eu' at /home/compiling-ganesh/24m0797/.cache/huggingface/datasets/xtreme/PAN-X.eu/0.0.0/ec5f1f46e9af79639a90684a7a70a956c4998f04 (last modified on Sat Dec 13 20:21:00 2025).


In [12]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

device = 'cuda'
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels = ds['train'].features['ner_tags'].feature.num_classes).to(device)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
def preprocess(item):
    res = tokenizer(item['tokens'], is_split_into_words=True, truncation=True)
    res['labels'] = []
    for i in range(len(res['input_ids'])):
        word_ids = res.word_ids(batch_index = i)
        last_id = None
        labels = []
        for word_id in word_ids:
            if word_id is not None and word_id != last_id:
                last_id = word_id
                labels.append(int(item['ner_tags'][i][word_id]))
            else:
                labels.append(-100)
            last_id = word_id
        res['labels'].append(labels)
    return res

tokenized_ds = ds.map(preprocess, batched = True)
item = tokenized_ds['train'][0]
item

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

{'tokens': ['BIRZUZENDU', 'Gertaera', 'arraroen', 'lege'],
 'ner_tags': [0, 3, 4, 4],
 'langs': ['eu', 'eu', 'eu', 'eu'],
 'input_ids': [101,
  12170,
  15378,
  20395,
  4859,
  2226,
  16216,
  13320,
  6906,
  12098,
  19848,
  8913,
  2078,
  4190,
  2063,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100,
  0,
  -100,
  -100,
  -100,
  -100,
  3,
  -100,
  -100,
  4,
  -100,
  -100,
  -100,
  4,
  -100,
  -100]}

In [14]:
tokenizer.decode(item['input_ids'])

'[CLS] birzuzendu gertaera arraroen lege [SEP]'

In [5]:
import pandas as pd

def pretty_print(item):
    breakpoint()
    text = item['tokens']
    tags = [ds['train'].features['ner_tags'].feature.names[idx] for idx in item['labels'] if idx != -100]
    df = pd.DataFrame([text, tags])
    return df

pretty_print(tokenized_ds['train'][0])

,0,1,2,3,4,5,6,7,8,9,10
0,R.H.,Saunders,(,St.,Lawrence,River,),(,968,MW,)
1,B-ORG,I-ORG,O,B-ORG,I-ORG,I-ORG,O,O,O,O,O


In [19]:
from transformers import Trainer, DataCollatorForTokenClassification, TrainingArguments
import evaluate
import numpy as np

f1 = evaluate.load('f1')

def compute_metrics(res):
    predictions, labels = np.argmax(res.predictions, axis=-1), res.label_ids
    final_pred, final_labels = [], []
    for p, l in zip(predictions, labels):
        for pred, label in zip(p, l):
            if label != -100:
                final_pred.append(pred)
                final_labels.append(label)
    return {
        'micro-f1': f1.compute(predictions = final_pred, references = final_labels, average = 'micro')['f1'],
        'macro-f1': f1.compute(predictions = final_pred, references = final_labels, average = 'macro')['f1'],
        'weighted-f1': f1.compute(predictions = final_pred, references = final_labels, average = 'weighted')['f1']
    }

def model_init():
    return AutoModelForTokenClassification.from_pretrained(model_name, num_labels = ds['train'].features['ner_tags'].feature.num_classes).to(device)

data_collator = DataCollatorForTokenClassification(tokenizer = tokenizer)

args = TrainingArguments(
    num_train_epochs = 3,
    per_device_train_batch_size = 32,
    output_dir = 'output/out',
    eval_steps = 10,
    logging_steps = 10,
    eval_strategy = 'steps'
)

trainer = Trainer(
    args = args,
    model_init = model_init,
    processing_class = tokenizer,
    data_collator = data_collator,
    train_dataset = tokenized_ds['train'],
    eval_dataset = tokenized_ds['test'].select(range(100)),
    compute_metrics = compute_metrics
)

trainer.train()

Using the latest cached version of the module from /home/compiling-ganesh/24m0797/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--f1/34c46321f42186df33a6260966e34a368f14868d9cc2ba47d142112e2800d233 (last modified on Tue Nov 25 11:27:07 2025) since it couldn't be found locally at evaluate-metric--f1, or remotely on the Hugging Face Hub.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

Step,Training Loss,Validation Loss,Micro-f1,Macro-f1,Weighted-f1
10,1.165900,0.942560,0.699355,0.317481,0.618143
20,0.695100,0.656212,0.805161,0.521845,0.770042
30,0.529700,0.480691,0.869677,0.708992,0.864276
40,0.420400,0.409959,0.882581,0.767866,0.883117
50,0.321900,0.306938,0.908387,0.812610,0.907654
60,0.294700,0.264469,0.913548,0.818285,0.912335
70,0.258700,0.237200,0.926452,0.853028,0.926081
80,0.238300,0.232744,0.921290,0.841133,0.921163
90,0.187700,0.235973,0.929032,0.860021,0.928879
100,0.188300,0.209547,0.935484,0.871794,0.934498


/home/compiling-ganesh/24m0797/workspace/transformers-from-scratch/.venv/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/compiling-ganesh/24m0797/workspace/transformers-from-scratch/.venv/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/compiling-ganesh/24m0797/workspace/transformers-from-scratch/.venv/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/compiling-ganesh/24m0797/workspace/transformers-from-scratch/.venv/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:70: U

TrainOutput(global_step=237, training_loss=0.260891642751573, metrics={'train_runtime': 43.3922, 'train_samples_per_second': 691.369, 'train_steps_per_second': 5.462, 'total_flos': 2311021146378912.0, 'train_loss': 0.260891642751573, 'epoch': 3.0})